In [2]:
!pwd
# !ls

/home/dephinate/ASU/DL/MedicalChatBot


In [3]:
import os
from pathlib import Path

In [4]:
# # GPU llama-cpp-python
# !CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip -I install llama-cpp-python numpy --force-reinstall --upgrade --no-cache-dir --verbose
# !pip install huggingface_hub
# !pip install -I llama-cpp-python
# !pip install numpy
# !pip install accelerate

In [5]:
# !pip install ctransformers
# !pip install sentence-transformers
# !pip install pinecone-client
# !pip install langchain
# !pip install langchain_pinecone
# !pip install flask
# !pip install pypdf
# !pip install ctransformers[cuda]

Using CTransformers

In [6]:
from langchain.llms import CTransformers

In [7]:
llm=CTransformers(model="llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

/home/dephinate/miniconda3/envs/medicalChatBot_recent/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using LlamaCpp

In [8]:
from langchain_community.llms import LlamaCpp

In [9]:
# GPU
lcpp_llm = None
lcpp_llm = LlamaCpp(
    model_path="llama-2-7b-chat.Q5_K_M.gguf",
    n_gpu_layers=32,
    n_batch=512,
    n_ctx=1024,
    f16_kv=True, 
    temperature = 0.8
    )

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from llama-2-7b-chat.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32              = 

Create Embeddings

In [10]:
from langchain.embeddings import HuggingFaceEmbeddings

In [11]:
# fucntion to download hugging face embeddings
def download_embeddings_from_huggingface(model_name:str):
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    return embeddings

In [12]:
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = download_embeddings_from_huggingface(model_name=embedding_model)

Pinecone

In [13]:
# Initialize index
from pinecone import Pinecone
pc = Pinecone(
    api_key = "ce80aa7c-c98e-467c-a100-b4b7e6a07c05",
)
pc.list_indexes().names() # just to doublecheck if I am able to connect to my Index
index_name = "medical-chatbot"
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.09221,
 'namespaces': {'medicalChatBot': {'vector_count': 9221}},
 'total_vector_count': 9221}

In [14]:
# Create Vectorstore
from langchain_pinecone import PineconeVectorStore
# docs_chunks =[t.page_content for t in chunks]

vectorstore = PineconeVectorStore(
    index=index,
    embedding=embeddings,
    namespace="medicalChatBot",
    index_name='medical-chatbot'
)

In [15]:
# Format Docs from Vectorstore
def format_docs(docs):
    c = 0
    return "\n".join(doc.page_content for doc in docs)

In [16]:
def format_docs(docs):
    numbered_docs = []
    for i, doc in enumerate(docs, start=1):  # Enumerate starting from 1
        numbered_docs.append(f"{i}. {doc.page_content}")  # Add number and content
    return "\n".join(numbered_docs)  # Join with newlines


In [1]:
query = "What is ACne ?"

chunks_retrieved = vectorstore.similarity_search(
    query,  # our search query
    k=3  # return 3 most relevant docs
)
chunks_retrieved_formatted = format_docs(chunks_retrieved)
print("Type",type(chunks_retrieved_formatted))
print("Len",len(chunks_retrieved_formatted))
print(chunks_retrieved_formatted)

NameError: name 'vectorstore' is not defined

In [18]:
chunks_retrieved

[Document(page_content='Acne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when thepores of the skin become clogged with oil, dead skincells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne, is'),
 Document(page_content='Nancy Ross-FlaniganKEY TERMS\nAcne —A skin condition in which raised bumps,\npimples, and cysts form on the face, neck, shoul-ders and upper back.\nBacteria —Tiny, one-celled forms of life that cause\nmany diseases and infections.\nBowel —The intestine; a tube-like structure that'),
 Document(page_content='Moderate and severe inflammatory types of acne\nresult after the plugged follicle is invaded by Propioni-\nbacterium acnes , a bacteria that normally lives on theskin. A pimple forms when the damaged follicle weakens')]

Prompting

In [19]:
# Template String
template1="""
[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>
Use the Context to answer the Question.
Context: {context}
Question: {question}[/INST]

"""

In [20]:
template2="""
Answer the question based on the context below.
If you don't know the answer, only return I don't know, Do not make an answer.

Context: {context}
Question: {question}

Answer:
"""

In [21]:
template3 = """
<<USER>> {question}

<<SYS>>
Answer the question based on the context below..
{context}
system_prompt: "You are a helpful assistant, below is ... llama-2-7b-chat.Q4_K_M.gguf"
"""

In [4]:
# Template String
template4="""
[INST] <<SYS>>
Answer the question based on the information. If you don't know the answer, only return 'I don't know', Do not make an answer.
<</SYS>>
question:{question}
information:{context}[/INST]

"""

In [49]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate.from_template(template4)
prompt

PromptTemplate(input_variables=['context', 'question'], template="\n[INST] <<SYS>>\nAnswer the question based on the context.If you don't know the answer, only return I don't know, Do not make an answer.\n<</SYS>>\nquestion:{question}\ncontext:{context}[/INST]\n\n")

In [51]:
from langchain.chains import RetrievalQA

In [5]:
qa=RetrievalQA.from_chain_type(
    llm=lcpp_llm, 
    chain_type="stuff", 
    retriever=vectorstore.as_retriever(search_kwargs={'k': 3}),
    return_source_documents=True, 
    chain_type_kwargs={"prompt": prompt})

NameError: name 'RetrievalQA' is not defined

In [53]:
response = qa.invoke({"query": "What is Acne"})


Llama.generate: prefix-match hit

llama_print_timings:        load time =   17519.10 ms
llama_print_timings:      sample time =      42.34 ms /    73 runs   (    0.58 ms per token,  1724.30 tokens per second)
llama_print_timings: prompt eval time =   21877.11 ms /   287 tokens (   76.23 ms per token,    13.12 tokens per second)
llama_print_timings:        eval time =   37569.36 ms /    72 runs   (  521.80 ms per token,     1.92 tokens per second)
llama_print_timings:       total time =   59857.75 ms /   359 tokens


In [54]:
response['source_documents']


[Document(page_content='Acne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when thepores of the skin become clogged with oil, dead skincells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne, is'),
 Document(page_content='Nancy Ross-FlaniganKEY TERMS\nAcne —A skin condition in which raised bumps,\npimples, and cysts form on the face, neck, shoul-ders and upper back.\nBacteria —Tiny, one-celled forms of life that cause\nmany diseases and infections.\nBowel —The intestine; a tube-like structure that'),
 Document(page_content='Moderate and severe inflammatory types of acne\nresult after the plugged follicle is invaded by Propioni-\nbacterium acnes , a bacteria that normally lives on theskin. A pimple forms when the damaged follicle weakens')]

In [55]:
response['result']

'Acne is a skin condition characterized by pimples, bumps, and cysts that occur on the face, neck, shoulders, and upper back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria. The medical term for acne is Acne vulgaris.'

M2

In [41]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(template4)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="\n[INST] <<SYS>>\nAnswer the question based on the context.If you don't know the answer, only return I don't know, Do not make an answer.\n<</SYS>>\nquestion:{question}\ncontext:{context}[/INST]\n\n"))])

In [42]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema.output_parser import StrOutputParser

In [43]:
retrieval = RunnableParallel(
    {"context": vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2}) | format_docs, "question": RunnablePassthrough()}
)

chain = retrieval | prompt | lcpp_llm | StrOutputParser()

In [44]:
retrieval.invoke("Caffien?")

{'context': '1. The caffeine acts by constricting blood vessels to relievethe headache . Sometimes, an analgesic such as aceta-\nminophen , caffeine, and a barbiturate which acts as a\n2. • reducing or stopping consumption of caffeine, alcohol,\nor tobacco; and/or\n• discontinuing diet pills or other medications (over-the-\ncounter or prescription)\nResources\nBOOKS\nCurrent Medical Diagnosis and Treatment, 1998. 37th ed. Ed.\nStephen McPhee, et al. Stamford: Appleton & Lange,1997.\nDeBakey, Michael E., and Antonio M. Gotto Jr. The New Liv-',
 'question': 'Caffien?'}

In [56]:
(retrieval|prompt).invoke("what is Acne and its diffent types and treatment?")

StringPromptValue(text="\n[INST] <<SYS>>\nAnswer the question based on the context.If you don't know the answer, only return I don't know, Do not make an answer.\n<</SYS>>\nquestion:what is Acne and its diffent types and treatment?\ncontext:1. Acne treatment consists of reducing sebum produc-\ntion, removing dead skin cells, and killing bacteria withtopical drugs and oral medications. Treatment choicedepends upon whether the acne is mild, moderate, orsevere.\nDrugs\nTOPICAL DRUGS. Treatment for mild noninflamma-\n2. Acne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when thepores of the skin become clogged with oil, dead skincells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne, is[/INST]\n\n")

In [57]:

result = chain.invoke("what is Acne and its diffent types and treatment?")
result

Llama.generate: prefix-match hit

llama_print_timings:        load time =   17519.10 ms
llama_print_timings:      sample time =     110.15 ms /   256 runs   (    0.43 ms per token,  2324.06 tokens per second)
llama_print_timings: prompt eval time =   16386.90 ms /   227 tokens (   72.19 ms per token,    13.85 tokens per second)
llama_print_timings:        eval time =  109442.39 ms /   255 runs   (  429.19 ms per token,     2.33 tokens per second)
llama_print_timings:       total time =  126737.09 ms /   482 tokens


'Based on the provided context, here are the different types of acne and their treatments:\nTypes of Acne:\n1. Mild acne: This type of acne is characterized by blackheads and whiteheads. Treatment for mild acne includes topical drugs such as benzoyl peroxide, salicylic acid, and sulfur.\n2. Moderate acne: This type of acne is characterized by papules, pustules, and nodules. Treatment for moderate acne includes topical drugs such as retinoids, oral antibiotics, and hormonal treatments.\n3. Severe acne: This type of acne is characterized by cysts and abscesses. Treatment for severe acne includes oral antibiotics, hormonal treatments, and Isotretinoin (a powerful oral medication).\nTopical Drugs:\nTopical drugs are applied directly to the skin to reduce sebum production, remove dead skin cells, and kill bacteria that cause acne. Some common topical drugs used to treat acne include:\n'

In [47]:
print(result)

Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria.


In [58]:
lcpp_llm.invoke("what is Acne and its diffent types and treatment?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =   17519.10 ms
llama_print_timings:      sample time =     110.62 ms /   256 runs   (    0.43 ms per token,  2314.19 tokens per second)
llama_print_timings: prompt eval time =    2328.33 ms /    12 tokens (  194.03 ms per token,     5.15 tokens per second)
llama_print_timings:        eval time =   84516.55 ms /   255 runs   (  331.44 ms per token,     3.02 tokens per second)
llama_print_timings:       total time =   87661.71 ms /   267 tokens


'\nWhat is Acne?Acne is a common skin condition that occurs when the pores on the skin become clogged with dead skin cells, oil, and bacteria. It can cause whiteheads, blackheads, and inflamed red pimples on the face, as well as other areas of the body. Acne can lead to emotional distress, social anxiety, and low self-esteem.\nTypes of acne:\nThere are several different types of acne, including:\n1. Comedonal acne: This type of acne occurs when dead skin cells and excess oil clog the pores, causing whiteheads and blackheads.\n2. Inflammatory acne: This type of acne occurs when a clogged pore becomes infected, leading to redness, swelling, and pus-filled pimples.\n3. Cystic acne: This is the most severe form of acne, causing large, painful cysts that can lead to scarring.\nTreatment for acne:\nTreatment for acne depends on the severity of the condition and may include:\n1.'

In [32]:
print(result)

3. Treatments for acne include over-the-counter (OTC)
medications and prescription medications. OTC treat-ments include benzoyl peroxide, salicylic acid, and sulfur. Prescription treatments include antibiotics, retinoids, and hormone therapy.
4. Avoid picking or squeezing acne as it can lead to infection and
scarring.
5. Maintain good hygiene habits such as washing your face twice daily with a mild soap and lukewarm water, and using non-comedogenic products to help prevent acne.
6. Eating a healthy diet rich in fruits, vegetables, whole grains, and lean protein can help to reduce acne.
7. Stress can trigger or worsen acne, so finding ways to manage stress such as exercise, meditation or deep breathing can be helpful.
8. Getting enough sleep can also help to reduce acne as lack of sleep can lead to increased stress levels which can contribute to acne.
9. Avoid touching your face as it can transfer bacteria
